In [1]:
import pandas as pd
import numpy as np

In [7]:
df_670 = pd.read_csv('ingredient_data_670.csv')
df_670 = df_670.iloc[:671,:]

In [11]:
df_671_2500 = pd.read_csv('ingredient_data_671_2500.csv')
df_671_2500 = df_671_2500.iloc[671:2501,:]

In [14]:
df_2501_2860 = pd.read_csv('ingredient_data_2501_2860.csv')
df_2501_2860 = df_2501_2860.iloc[2503:2861,:]

In [17]:
df_2500 = df_670.append(df_671_2500)
df_2860 = df_2500.append(df_2501_2860)

In [19]:
df_2860.to_csv('ingredient_2860.csv', index=False)

# Clean

In [20]:
df = pd.read_csv('ingredient_2860.csv')
# check num of recipes
df.shape

(2859, 30)

In [22]:
# check nan
df.isnull().sum()

link                   0
Beef                   0
Beans and Legumes      0
Chicken Recipes        0
Game Meats             0
Grains                 0
Mushrooms              0
Pasta                  0
Pork Recipes           0
Potatoes               0
Poultry                0
Rice                   0
Salmon                 0
Seafood                0
Shrimp                 0
Tofu and Tempeh        0
Turkey                 0
Vegetable Recipes      0
title                  0
ratings              428
num_reviews          429
ingredients            0
servings             429
cook_time            552
calorie              432
fat_g                432
carb_g               432
protein_g            432
cholesterol_mg       432
sodium_mg            433
dtype: int64

In [28]:
# check rows with no ratings
# df[df.ratings.isnull()]

In [24]:
# they also have other info missing so I will drop them
to_drop = df[df.ratings.isnull()].index.tolist()
df.drop(to_drop, inplace=True)

# check num of recipes
df.shape

(2431, 30)

In [25]:
# check rows with no nutrition info
# df[df.calorie.isnull()]

# I need nutrition info to generate meal plans so I will drop these rows
to_drop = df[df.calorie.isnull()].index.tolist()
df.drop(to_drop, inplace=True)

# check num of recipes
df.shape

(2427, 30)

In [26]:
# check nan again
df.isnull().sum()

link                   0
Beef                   0
Beans and Legumes      0
Chicken Recipes        0
Game Meats             0
Grains                 0
Mushrooms              0
Pasta                  0
Pork Recipes           0
Potatoes               0
Poultry                0
Rice                   0
Salmon                 0
Seafood                0
Shrimp                 0
Tofu and Tempeh        0
Turkey                 0
Vegetable Recipes      0
title                  0
ratings                0
num_reviews            0
ingredients            0
servings               0
cook_time            123
calorie                0
fat_g                  0
carb_g                 0
protein_g              0
cholesterol_mg         0
sodium_mg              1
dtype: int64

In [27]:
# check data types
df.dtypes

link                  object
Beef                   int64
Beans and Legumes      int64
Chicken Recipes        int64
Game Meats             int64
Grains                 int64
Mushrooms              int64
Pasta                  int64
Pork Recipes           int64
Potatoes               int64
Poultry                int64
Rice                   int64
Salmon                 int64
Seafood                int64
Shrimp                 int64
Tofu and Tempeh        int64
Turkey                 int64
Vegetable Recipes      int64
title                 object
ratings              float64
num_reviews           object
ingredients           object
servings             float64
cook_time             object
calorie               object
fat_g                float64
carb_g               float64
protein_g            float64
cholesterol_mg        object
sodium_mg             object
dtype: object

In [10]:
# change data types of num_reviews, cook_time, calorie, cholesterol_mg and sodium_mg 
# to int or float and fill missing values 

# check the row with missing num_reviews
df_breakfast[df_breakfast.num_reviews.isnull()]

,link,Breakfast and Brunch,Dinners,Lunch,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
573,https://www.allrecipes.com/recipe/231244/aspar...,1,0,0,asparagus-mushroom-bacon-crustless-quiche,4.752809,NaN,"['6 ounces bacon, diced', '1/2 pound asparagus...",8.0,55 m,291 calories;,25.9,3.9,11.5,175,315.0


In [11]:
# replace missing value of num_reviews with mode
df_breakfast.num_reviews.fillna(df_breakfast.num_reviews.mode()[0], inplace=True)

# check the result
df_breakfast.loc[573]

link                    https://www.allrecipes.com/recipe/231244/aspar...
Breakfast and Brunch                                                    1
Dinners                                                                 0
Lunch                                                                   0
title                           asparagus-mushroom-bacon-crustless-quiche
ratings                                                           4.75281
num_reviews                                                    23 reviews
ingredients             ['6 ounces bacon, diced', '1/2 pound asparagus...
servings                                                                8
cook_time                                                            55 m
calorie                                                     291 calories;
fat_g                                                                25.9
carb_g                                                                3.9
protein_g                             

### Clean num_reviews

In [29]:
df.head(1)

,link,Beef,Beans and Legumes,Chicken Recipes,Game Meats,Grains,Mushrooms,Pasta,Pork Recipes,Potatoes,...,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100182/grand...,1,0,0,0,0,0,0,0,0,...,371 reviews,"['2 pounds ground beef', '1/2 cup chopped onio...",12.0,2 h 10 m,186 calories;,9.2,12.8,13.5,46,586


In [30]:
# for num_reviews, remove " reviews"
df.num_reviews = df.num_reviews.apply(lambda x: x[:-8])

In [31]:
# some num_reviews contain "k", I will replace them with "000"
df_replace_k = df[df.num_reviews.str.contains('k')]
df_replace_k.num_reviews = df_replace_k.num_reviews.apply(lambda x: x[:-1] + '000')

# update rows in the original df
replace_idx = df_replace_k.index.tolist()
for idx in replace_idx: 
    df.loc[idx] = df_replace_k.loc[idx]

# change data type to int
df.num_reviews = df.num_reviews.astype('int')

# check results
print(df.num_reviews.dtypes)
df.loc[replace_idx[:5]]

/Users/fivecentsly/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


int64


,link,Beef,Beans and Legumes,Chicken Recipes,Game Meats,Grains,Mushrooms,Pasta,Pork Recipes,Potatoes,...,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
14,https://www.allrecipes.com/recipe/10141/outrag...,0,0,0,0,1,0,0,0,0,...,2000,"['1/2 cup butter', '1/2 cup white sugar', '1/3...",18.0,25 m,207 calories;,12.0,23.7,3.6,24,178
19,https://www.allrecipes.com/recipe/10264/oatmea...,0,0,0,0,1,0,0,0,0,...,1000,"['3/4 cup butter, softened', '3/4 cup white su...",48.0,1 h,92 calories;,3.4,14.6,1.3,15,75
23,https://www.allrecipes.com/recipe/10308/oatmea...,0,0,0,0,1,0,0,0,0,...,2000,"['1/2 cup shortening', '1/2 cup margarine, sof...",48.0,1 h,120 calories;,7.1,12.8,2.3,8,152
43,https://www.allrecipes.com/recipe/104850/black...,0,1,0,0,0,0,0,0,0,...,1000,"['2 teaspoons olive oil', '3 tablespoons finel...",8.0,40 m,363 calories;,14.5,45.6,13.9,26,732
48,https://www.allrecipes.com/recipe/10497/beths-...,0,0,0,0,1,0,0,0,0,...,3000,"['1/2 cup butter, softened', '1/2 cup butter f...",36.0,50 m,144 calories;,6.3,20.6,1.9,17,92


### Clean cook_time

In [32]:
# for cook_time, subset the rows with no missing values
df_cooktime = df[df.cook_time.notnull()]

In [33]:
# define a function to convert cook_time
def convert_cooktime(x):
    cooktime = x.split()
    if 'h' in cooktime and 'm' in cooktime:
        x = int(cooktime[0]) * 60 + int(cooktime[2])
    elif 'h' in cooktime and 'm' not in cooktime:
        x = int(cooktime[0]) * 60
    else:
        x = int(cooktime[0])   
    return x

In [34]:
# apply function to cook_time
df_cooktime.cook_time = df_cooktime.cook_time.apply(convert_cooktime)

# update rows in the original df
replace_idx = df_cooktime.index.tolist()
for idx in replace_idx: 
    df.loc[idx] = df_cooktime.loc[idx]

# check results
print(df.cook_time.dtypes) # data type is object because there are missing values
df.loc[replace_idx[:5]]

object


,link,Beef,Beans and Legumes,Chicken Recipes,Game Meats,Grains,Mushrooms,Pasta,Pork Recipes,Potatoes,...,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100182/grand...,1,0,0,0,0,0,0,0,0,...,371,"['2 pounds ground beef', '1/2 cup chopped onio...",12.0,130,186 calories;,9.2,12.8,13.5,46,586
1,https://www.allrecipes.com/recipe/100195/baked...,0,0,0,0,0,0,0,0,0,...,732,"['3/4 cup milk', '2 teaspoons salt', '3/4 cup ...",4.0,25,325 calories;,15.7,17.0,27.7,103,1565
2,https://www.allrecipes.com/recipe/100218/corri...,0,1,0,0,0,0,0,0,0,...,134,"['2 tablespoons olive oil', '5 potatoes, peele...",12.0,60,221 calories;,3.1,41.8,7.5,0,874
4,https://www.allrecipes.com/recipe/100295/irish...,0,0,0,0,0,0,0,0,1,...,122,"['2 pounds potatoes, peeled and halved', '1 cu...",4.0,40,326 calories;,13.0,46.8,7.8,35,420
5,https://www.allrecipes.com/recipe/100402/quick...,0,0,0,0,0,0,0,0,0,...,52,"['2 (6 ounce) cans pink salmon, drained', '1/2...",4.0,15,248 calories;,17.2,2.2,20.4,43,571


In [36]:
# replace missing values of cook_time with 0 to indicate "unknown"
df.cook_time.fillna(0, inplace=True)
df.cook_time.dtypes

dtype('int64')

### Clean calorie

In [37]:
# for calorie, only keep the number and change data type to float
df.calorie = df.calorie.apply(lambda x: float(x.split(' ')[0]))

# check results 
print(df.calorie.dtypes)
df.head(1)

float64


,link,Beef,Beans and Legumes,Chicken Recipes,Game Meats,Grains,Mushrooms,Pasta,Pork Recipes,Potatoes,...,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100182/grand...,1,0,0,0,0,0,0,0,0,...,371,"['2 pounds ground beef', '1/2 cup chopped onio...",12.0,130,186.0,9.2,12.8,13.5,46,586


### Clean cholesterol_mg and sodium_mg

In [38]:
# some values of cholesterol_mg and sodium_mg are shown as "< 1", I will replace them with "0"
df.cholesterol_mg = df.cholesterol_mg.replace('< 1', 0)
df.sodium_mg = df.sodium_mg.replace('< 1', 0)

# change their data types to float
df.cholesterol_mg = df.cholesterol_mg.astype(float)
df.sodium_mg = df.sodium_mg.astype(float)

# check results 
print(df.cholesterol_mg.dtypes)
print(df.sodium_mg.dtypes)

float64
float64


In [49]:
# fill missing value of sodium_mg with mean
df.sodium_mg = df.sodium_mg.fillna(np.round(df.sodium_mg.mean()))

### Clean ratings

In [56]:
# round ratings to two decimals
df.ratings = df.ratings.round(2)

# check results
df.head(5)['ratings']

0    4.29
1    4.59
2    4.24
3    4.67
4    4.62
Name: ratings, dtype: float64

### Clean title

In [57]:
# change the format of title
df.title = df.title.apply(lambda x: x.replace('-', ' ').title())

# check results
df.head(5)['title']

0    Grandpas Classic Coney Sauce
1                   Baked Haddock
2            Corrigans Minestrone
3     Mrs Fields Cookie Recipe Ii
4                     Irish Champ
Name: title, dtype: object

In [58]:
# save the clean data
df.to_csv('ingredient_2860_clean.csv', index=False)

In [25]:
# build a clean function to apply to lunch_data and dinner_data
def convert_cooktime(x):
    cooktime = x.split()
    if 'h' in cooktime and 'm' in cooktime:
        x = int(cooktime[0]) * 60 + int(cooktime[2])
    elif 'h' in cooktime and 'm' not in cooktime:
        x = int(cooktime[0]) * 60
    else:
        x = int(cooktime[0])   
    return x

def clean_data(df):
    """
    Clean df
    """
    # clean num_reviews
    print('Cleaning num_reviews...')
    df.num_reviews = df.num_reviews.apply(lambda x: x[:-8])
    df_replace_k = df[df.num_reviews.str.contains('k')]
    df_replace_k.num_reviews = df_replace_k.num_reviews.apply(lambda x: x[:-1] + '000')

    # update rows in the original df
    replace_idx = df_replace_k.index.tolist()
    for idx in replace_idx: 
        df.loc[idx] = df_replace_k.loc[idx]

    # change data type to int
    df.num_reviews = df.num_reviews.astype('int')
    
    # clean cook_time
    print('Cleaning cook_time...')
    df_cooktime = df[df.cook_time.notnull()]
    df_cooktime.cook_time = df_cooktime.cook_time.apply(convert_cooktime)

    # update rows in the original df
    replace_idx = df_cooktime.index.tolist()
    for idx in replace_idx: 
        df.loc[idx] = df_cooktime.loc[idx]
    
    # replace missing values of cook_time with 0 to indicate "unknown"
    df.cook_time.fillna(0, inplace=True)

    # clean calorie
    print('Cleaning calorie, cholesterol_mg and sodium_mg...')
    df.calorie = df.calorie.apply(lambda x: float(x.split(' ')[0]))
    
    # clean cholesterol_mg and sodium_mg 
    df.cholesterol_mg = df.cholesterol_mg.replace('< 1', 0)
    df.sodium_mg = df.sodium_mg.replace('< 1', 0)

    # change their data types to float
    df.cholesterol_mg = df.cholesterol_mg.astype(float)
    df.sodium_mg = df.sodium_mg.astype(float)
    
    # clean ratings 
    print('Cleaning ratings and title...')
    df.ratings = df.ratings.round(2)
          
    # clean title
    df.title = df.title.apply(lambda x: x.replace('-', ' ').title())
          
    return df

In [55]:
# load lunch data
df_lunch = pd.read_csv('recipe_data/lunch_data.csv')
# check num of recipes
df_lunch.shape

(1252, 16)

In [56]:
# check nan
df_lunch.isnull().sum()

link                      0
Breakfast and Brunch      0
Dinners                   0
Lunch                     0
title                     1
ratings                 231
num_reviews             231
ingredients               1
servings                231
cook_time               247
calorie                 235
fat_g                   235
carb_g                  235
protein_g               235
cholesterol_mg          235
sodium_mg               235
dtype: int64

In [57]:
# check rows with no ratings
df_lunch[df_lunch.ratings.isnull()]

,link,Breakfast and Brunch,Dinners,Lunch,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
3,https://www.allrecipes.com/recipe/100546/japan...,0,0,1,japanese-tofu-salad,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.allrecipes.com/recipe/100593/chick...,0,0,1,chicken-noodle-salad-with-peanut-ginger-dressing,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://www.allrecipes.com/recipe/100612/chedd...,0,0,1,cheddar-baby-leek-and-tomato-sandwich,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,https://www.allrecipes.com/recipe/106381/asian...,0,0,1,asian-pasta-salad-with-beef-broccoli-and-bean-...,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,https://www.allrecipes.com/recipe/106515/peruv...,0,0,1,peruvian-chicken-soup-aguadito-de-pollo,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,https://www.allrecipes.com/recipe/108422/chick...,0,0,1,chicken-and-bacon-fajitas,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,https://www.allrecipes.com/recipe/108540/kentu...,0,0,1,kentucky-tomato-soup,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,https://www.allrecipes.com/recipe/109740/sourd...,0,0,1,sourdough-chipotle-chicken-panini,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,https://www.allrecipes.com/recipe/112133/best-...,0,0,1,best-baconless-broccoli-salad,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,https://www.allrecipes.com/recipe/128589/oyako...,0,0,1,oyakodon-japanese-chicken-and-egg-rice-bowl,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# they also have other info missing so I will drop them
to_drop = df_lunch[df_lunch.ratings.isnull()].index.tolist()
df_lunch.drop(to_drop, inplace=True)

# check num of recipes
df_lunch.shape

(1021, 16)

In [59]:
# I need nutrition info to generate meal plans so I will drop these rows with missing nutrition info
to_drop = df_lunch[df_lunch.calorie.isnull()].index.tolist()
df_lunch.drop(to_drop, inplace=True)

# check num of recipes
df_lunch.shape

(1017, 16)

In [60]:
# check nan again
df_lunch.isnull().sum()

link                     0
Breakfast and Brunch     0
Dinners                  0
Lunch                    0
title                    0
ratings                  0
num_reviews              0
ingredients              0
servings                 0
cook_time               16
calorie                  0
fat_g                    0
carb_g                   0
protein_g                0
cholesterol_mg           0
sodium_mg                0
dtype: int64

In [61]:
# check data types
df_lunch.dtypes

link                     object
Breakfast and Brunch      int64
Dinners                   int64
Lunch                     int64
title                    object
ratings                 float64
num_reviews              object
ingredients              object
servings                float64
cook_time                object
calorie                  object
fat_g                   float64
carb_g                  float64
protein_g               float64
cholesterol_mg           object
sodium_mg                object
dtype: object

In [62]:
# apply clean_data
clean_data(df_lunch)

Cleaning num_reviews...
Cleaning cook_time...
Cleaning calorie, cholesterol_mg and sodium_mg...
Cleaning ratings and title...


,link,Breakfast and Brunch,Dinners,Lunch,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100164/chip-...,0,0,1,Chip Truck Fries,4.44,44,"['6 baking potatoes, cut into 1/4 inch slices'...",4.0,60,369.0,14.0,56.3,6.5,0.0,478.0
1,https://www.allrecipes.com/recipe/100378/irish...,0,0,1,Irish Bacon And Cabbage Soup,4.54,163,"['1/2 pound Irish bacon, diced', '2 large pota...",4.0,45,276.0,8.1,38.4,12.3,21.0,825.0
2,https://www.allrecipes.com/recipe/100402/quick...,0,0,1,Quick Salmon Salad,4.49,52,"['2 (6 ounce) cans pink salmon, drained', '1/2...",4.0,15,248.0,17.2,2.2,20.4,43.0,571.0
6,https://www.allrecipes.com/recipe/102242/bills...,0,0,1,Bills Seasoned Ketchup,4.65,22,"['1 (28 ounce) bottle ketchup', '2 teaspoons h...",45.0,10,19.0,0.1,4.7,0.4,0.0,201.0
7,https://www.allrecipes.com/recipe/103737/celer...,0,0,1,Celery Salad,4.59,83,"['3/4 cup sliced celery', '1/3 cup dried sweet...",2.0,40,150.0,3.0,26.5,4.1,0.0,304.0
8,https://www.allrecipes.com/recipe/103767/jens-...,0,0,1,Jens Heavenly Egg Salad,4.47,307,"['6 eggs', '1/4 cup mayonnaise', '1 teaspoon D...",4.0,21,212.0,18.4,3.2,9.9,284.0,223.0
9,https://www.allrecipes.com/recipe/104850/black...,0,0,1,Black Bean And Corn Quesadillas,4.62,1000,"['2 teaspoons olive oil', '3 tablespoons finel...",8.0,40,363.0,14.5,45.6,13.9,26.0,732.0
10,https://www.allrecipes.com/recipe/104928/mimis...,0,0,1,Mimis Zucchini Pie,4.40,136,"['4 eggs', '1/4 cup vegetable oil', 'salt and ...",8.0,60,182.0,12.4,12.6,6.0,95.0,400.0
11,https://www.allrecipes.com/recipe/105581/tomat...,0,0,1,Tomato Mint Quinoa Salad,4.49,67,"['2 1/2 cups water', '1 1/4 cups quinoa', '1/3...",4.0,115,346.0,11.8,53.3,9.5,0.0,178.0
12,https://www.allrecipes.com/recipe/106221/nutty...,0,0,1,Nutty Brown Rice Salad,4.47,42,"['1 cup uncooked brown rice', '1 1/2 cups wate...",6.0,150,223.0,4.5,39.2,7.1,0.0,203.0


In [44]:
df_lunch.dtypes

link                     object
Breakfast and Brunch      int64
Dinners                   int64
Lunch                     int64
title                    object
ratings                 float64
num_reviews               int64
ingredients              object
servings                float64
cook_time                 int64
calorie                 float64
fat_g                   float64
carb_g                  float64
protein_g               float64
cholesterol_mg          float64
sodium_mg               float64
dtype: object

In [45]:
df_lunch.to_csv('lunch_clean.csv', index=False)

In [52]:
# load dinner data
df_dinner = pd.read_csv('recipe_data/dinner_data.csv')
# check num of recipes
df_dinner.shape

(1296, 16)

In [53]:
# check nan
df_dinner.isnull().sum()

link                      0
Breakfast and Brunch      0
Dinners                   0
Lunch                     0
title                     1
ratings                 330
num_reviews             331
ingredients               3
servings                331
cook_time               357
calorie                 337
fat_g                   337
carb_g                  337
protein_g               337
cholesterol_mg          337
sodium_mg               337
dtype: int64

In [54]:
# check rows with no ratings
df_dinner[df_dinner.ratings.isnull()]

,link,Breakfast and Brunch,Dinners,Lunch,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
3,https://www.allrecipes.com/recipe/102803/grand...,0,1,0,grandmas-bogies-parmesan-chicken,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,https://www.allrecipes.com/recipe/106528/sour-...,0,1,0,sour-cream-enchiladas,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,https://www.allrecipes.com/recipe/109925/pobla...,0,1,0,poblano-chile-enchiladas-a-la-gringa,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,https://www.allrecipes.com/recipe/11749/chicke...,0,1,0,chicken-pesto-a-la-lisa,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,https://www.allrecipes.com/recipe/12009/creamy...,0,1,0,creamy-cajun-chicken-pasta,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,https://www.allrecipes.com/recipe/125658/chick...,0,1,0,chicken-enchiladas-with-creamy-green-chile-sauce,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,https://www.allrecipes.com/recipe/12620/low-fa...,0,1,0,low-fat-chicken-pot-pie,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,https://www.allrecipes.com/recipe/12651/chicke...,0,1,0,chicken-biscuit-pie,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,https://www.allrecipes.com/recipe/12659/chicke...,0,1,0,chicken-pot-pie-iii,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,https://www.allrecipes.com/recipe/12690/cheesy...,0,1,0,cheesy-chicken-pot-pie,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# they also have other info missing so I will drop them
to_drop = df_dinner[df_dinner.ratings.isnull()].index.tolist()
df_dinner.drop(to_drop, inplace=True)

# check num of recipes
df_dinner.shape

(966, 16)

In [64]:
# I need nutrition info to generate meal plans so I will drop these rows with missing nutrition info
to_drop = df_dinner[df_dinner.calorie.isnull()].index.tolist()
df_dinner.drop(to_drop, inplace=True)

# check num of recipes
df_dinner.shape

(959, 16)

In [65]:
# check nan again
df_dinner.isnull().sum()

link                     0
Breakfast and Brunch     0
Dinners                  0
Lunch                    0
title                    0
ratings                  0
num_reviews              0
ingredients              1
servings                 0
cook_time               26
calorie                  0
fat_g                    0
carb_g                   0
protein_g                0
cholesterol_mg           0
sodium_mg                0
dtype: int64

In [66]:
# I need ingredients to predict meal plans so I will drop the one with no ingredients
to_drop = df_dinner[df_dinner.ingredients.isnull()].index.tolist()
df_dinner.drop(to_drop, inplace=True)

In [67]:
# check data types
df_dinner.dtypes

link                     object
Breakfast and Brunch      int64
Dinners                   int64
Lunch                     int64
title                    object
ratings                 float64
num_reviews              object
ingredients              object
servings                float64
cook_time                object
calorie                  object
fat_g                   float64
carb_g                  float64
protein_g               float64
cholesterol_mg          float64
sodium_mg               float64
dtype: object

In [68]:
# I may not to clean cholesterol_mg and sodium_mg, I will revise the clean function for dinner_data
def clean_dinner(df):
    """
    Clean df
    """
    # clean num_reviews
    print('Cleaning num_reviews...')
    df.num_reviews = df.num_reviews.apply(lambda x: x[:-8])
    df_replace_k = df[df.num_reviews.str.contains('k')]
    df_replace_k.num_reviews = df_replace_k.num_reviews.apply(lambda x: x[:-1] + '000')

    # update rows in the original df
    replace_idx = df_replace_k.index.tolist()
    for idx in replace_idx: 
        df.loc[idx] = df_replace_k.loc[idx]

    # change data type to int
    df.num_reviews = df.num_reviews.astype('int')
    
    # clean cook_time
    print('Cleaning cook_time...')
    df_cooktime = df[df.cook_time.notnull()]
    df_cooktime.cook_time = df_cooktime.cook_time.apply(convert_cooktime)

    # update rows in the original df
    replace_idx = df_cooktime.index.tolist()
    for idx in replace_idx: 
        df.loc[idx] = df_cooktime.loc[idx]
    
    # replace missing values of cook_time with 0 to indicate "unknown"
    df.cook_time.fillna(0, inplace=True)

    # clean calorie
    print('Cleaning calorie...')
    df.calorie = df.calorie.apply(lambda x: float(x.split(' ')[0]))
    
    # clean ratings 
    print('Cleaning ratings and title...')
    df.ratings = df.ratings.round(2)
          
    # clean title
    df.title = df.title.apply(lambda x: x.replace('-', ' ').title())
          
    return df

In [69]:
clean_dinner(df_dinner)

Cleaning num_reviews...
Cleaning cook_time...
Cleaning calorie...
Cleaning ratings and title...


,link,Breakfast and Brunch,Dinners,Lunch,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100195/baked...,0,1,0,Baked Haddock,4.59,731,"['3/4 cup milk', '2 teaspoons salt', '3/4 cup ...",4.0,25,325.0,15.7,17.0,27.7,103.0,1565.0
1,https://www.allrecipes.com/recipe/100606/beef-...,0,1,0,Beef Bulgogi,4.79,664,"['1 pound flank steak, thinly sliced', '5 tabl...",4.0,75,232.0,13.2,12.4,16.2,27.0,1157.0
2,https://www.allrecipes.com/recipe/102159/apric...,0,1,0,Apricot Brown Sugar Ham,4.72,210,"['1 (10 pound) fully-cooked spiral cut ham', '...",15.0,120,801.0,56.2,14.4,56.1,170.0,3895.0
4,https://www.allrecipes.com/recipe/103503/taco-...,0,1,0,Taco Stuffed Shells,4.55,111,"['16 jumbo pasta shells', '1 pound ground beef...",8.0,80,361.0,17.6,30.4,19.8,60.0,957.0
5,https://www.allrecipes.com/recipe/103535/black...,0,1,0,Blackened Tuna Steaks With Mango Salsa,4.85,238,"['2 tablespoons olive oil', '2 tablespoons lim...",4.0,235,458.0,24.7,18.3,42.1,77.0,1233.0
6,https://www.allrecipes.com/recipe/103971/grill...,0,1,0,Grilled Halibut With Cilantro Garlic Butter,4.53,180,"['4 (6 ounce) fillets halibut', '1 lime, cut i...",4.0,33,276.0,13.1,3.0,35.4,69.0,174.0
7,https://www.allrecipes.com/recipe/103991/broil...,0,1,0,Broiled Grouper Parmesan,4.67,217,"['2 pounds fresh grouper fillets', '2 tablespo...",4.0,16,430.0,24.9,1.4,48.1,127.0,389.0
8,https://www.allrecipes.com/recipe/104621/barbe...,0,1,0,Barbequed Steelhead Trout,4.58,131,"['2 pounds steelhead trout fillets', '1/4 cup ...",6.0,20,285.0,16.2,4.3,28.8,101.0,221.0
9,https://www.allrecipes.com/recipe/104789/cocon...,0,1,0,Coconut Tilapia With Apricot Dipping Sauce,4.58,209,"['1 cup flaked coconut, finely chopped', '2 ta...",4.0,30,650.0,36.1,54.3,28.4,42.0,589.0
10,https://www.allrecipes.com/recipe/105044/lamb-...,0,1,0,Lamb Tagine,4.67,247,"['3 tablespoons olive oil, divided', '2 pounds...",4.0,645,423.0,20.5,23.6,35.8,109.0,1129.0


In [70]:
df_dinner.dtypes

link                     object
Breakfast and Brunch      int64
Dinners                   int64
Lunch                     int64
title                    object
ratings                 float64
num_reviews               int64
ingredients              object
servings                float64
cook_time                 int64
calorie                 float64
fat_g                   float64
carb_g                  float64
protein_g               float64
cholesterol_mg          float64
sodium_mg               float64
dtype: object

In [71]:
df_dinner.to_csv('dinner_clean.csv', index=False)